# TP2 - Sistemas Autónomos
#### "Monitorização de parâmetros ambientais para a prática de desporto"

## Import Libraries

In [ ]:
import sys
import config
import firebase_admin
from firebase_admin import auth, credentials, firestore
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt

## Get the Data


### Get the Database from Cloud Firestore

In [ ]:
# Get the key
cred = credentials.Certificate(config.firestore_key)

# Initialize/Get the app
try:
    app = firebase_admin.initialize_app(cred)
except:
    app = firebase_admin.get_app()

# Get the database
db = firestore.client()

### Get the 'WM' collection

In [ ]:
wm_ref = db.collection(u'WM')
docs = wm_ref.stream()

values_list = []

# general_weather tem que se meter em integer, mas temos que decidir se continuamos a extrair o "description" ou o "main" no momento do request

for doc in docs:
    params = doc.to_dict()
    datetime = re.split(r'[-T:]',doc.id)
    perc_minutes = int(datetime[3])+(0.0167*int(datetime[4])) # precisava de ter os minutos adaptados para a unidade da hora
    values_list.append([int(datetime[0]),int(datetime[1]),int(datetime[2]),int(datetime[3]),int(datetime[4]),perc_minutes,float(params['feels_like'])-273.15,params['general_weather'],int(params['humidity']),int(params['pressure']),float(params['temp_max'])-273.15,float(params['temp_min'])-273.15,float(params['wind_speed'])])

wm_data = pd.DataFrame(values_list,columns=['year','month','day','hours','minutes','hours.minutes','feels_like','general_weather','humidity','pressure','temp_max','temp_min','wind_speed'])

wm_data.head()

In [ ]:
wm_data.info()

### Get the 'UV' collection


In [ ]:
uv_ref = db.collection(u'UV')
docs = uv_ref.stream()

values_list = []

# falta tratar dos types de alguns parametros

for doc in docs:
    params = doc.to_dict()
    datetime = re.split(r'[-T:]',doc.id)
    perc_minutes = int(datetime[3])+(0.0167*int(datetime[4])) # precisava de ter os minutos adaptados para a unidade da hora
    values_list.append([int(datetime[0]),int(datetime[1]),int(datetime[2]),int(datetime[3]),int(datetime[4]),perc_minutes,float(params['uv']),params['uv_time'],float(params['uv_max']),params['uv_max_time'],params['st1'],params['st2'],params['st3'],params['st4'],params['st5'],params['st6']])

uv_data = pd.DataFrame(values_list,columns=['year','month','day','hours','minutes','hours.minutes','uv','uv_time','uv_max','uv_max_time','st1','st2','st3','st4','st5','st6'])

uv_data.head()

In [ ]:
uv_data.info()

### Get the 'AQ' collection

In [ ]:
aq_ref = db.collection(u'AQ')
docs = aq_ref.stream()

values_list = []

for doc in docs:
    params = doc.to_dict()
    datetime = re.split(r'[-T:]',doc.id)
    perc_minutes = int(datetime[3])+(0.0167*int(datetime[4]))# precisava de ter os minutos adaptados para a unidade da hora
    values_list.append([int(datetime[0]),int(datetime[1]),int(datetime[2]),int(datetime[3]),int(datetime[4]),perc_minutes,int(params['no2']),int(params['o3']),int(params['pm10'])])

aq_data = pd.DataFrame(values_list,columns=['year','month','day','hours','minutes','hours.minutes','no2','o3','pm10'])

aq_data.head()

In [ ]:
aq_data.info()

## Exploratory Data Analysis

In [ ]:
sns.set_palette("GnBu_d")
sns.set_style('whitegrid')

In [ ]:

sns.jointplot(x='hours.minutes', y='feels_like', data=wm_data)


In [ ]:
sns.jointplot(x='hours.minutes', y='uv', data=uv_data)


In [ ]:
sns.jointplot(x='hours.minutes', y='o3', data=aq_data)